# Timeseries Analysis of Appliance and Light Usage

## Frame the problem and look at the big picture

Please see the report on this project in the [repository](https://github.com/parksjr5/Energy_Forecasting).

## Get the data

#### Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import math
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import grangercausalitytests

In [ ]:
# import data
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00374/energydata_complete.csv')
df.head(3)

In [ ]:
# data dimensions
df.shape

In [ ]:
# data info
df.info()

#### Create train and test data

In [ ]:
nobs = int(.8*df.shape[0])
df_test, df_train = df[0:-nobs], df[-nobs:]
print(df_train.shape)
print(df_test.shape)

In [ ]:
# create exploratory data
exp_df = df_train

## Explore the data

#### Attribute and Characteristics

In [ ]:
# column names
exp_df.columns

In [ ]:
# check for missing values
exp_df.isna().sum()

In [ ]:
# check data types of each column
exp_df.dtypes

#### Visualize Data

In [ ]:
fig, ax = plt.subplots(4,5, figsize=(16,9))

ax[0,0].plot(exp_df['date'].iloc[:100,], exp_df['T1'].iloc[:100,])
ax[0,0].tick_params(labelbottom = False, bottom = False)
ax[0,0].set_ylabel('Temperature')
ax[1,0].plot(exp_df['date'].iloc[:100,], exp_df['RH_1'].iloc[:100,])
ax[1,0].tick_params(labelbottom = False, bottom = False)
ax[1,0].set_ylabel('Humidity')
ax[0,0].set_title('Kitchen Area')

ax[0,1].plot(exp_df['date'].iloc[:100,], exp_df['T2'].iloc[:100,])
ax[0,1].tick_params(labelbottom = False, bottom = False)
ax[1,1].plot(exp_df['date'].iloc[:100,], exp_df['RH_2'].iloc[:100,])
ax[1,1].tick_params(labelbottom = False, bottom = False)
ax[0,1].set_title('Living Area')

ax[0,2].plot(exp_df['date'].iloc[:100,], exp_df['T3'].iloc[:100,])
ax[0,2].tick_params(labelbottom = False, bottom = False)
ax[1,2].plot(exp_df['date'].iloc[:100,], exp_df['RH_3'].iloc[:100,])
ax[1,2].tick_params(labelbottom = False, bottom = False)
ax[0,2].set_title('Laundry Area')

ax[0,3].plot(exp_df['date'].iloc[:100,], exp_df['T4'].iloc[:100,])
ax[0,3].tick_params(labelbottom = False, bottom = False)
ax[1,3].plot(exp_df['date'].iloc[:100,], exp_df['RH_4'].iloc[:100,])
ax[1,3].tick_params(labelbottom = False, bottom = False)
ax[0,3].set_title('Office Area')

ax[0,4].plot(exp_df['date'].iloc[:100,], exp_df['T5'].iloc[:100,])
ax[0,4].tick_params(labelbottom = False, bottom = False)
ax[1,4].plot(exp_df['date'].iloc[:100,], exp_df['RH_5'].iloc[:100,])
ax[1,4].tick_params(labelbottom = False, bottom = False)
ax[0,4].set_title('Bathroom')

ax[2,0].plot(exp_df['date'].iloc[:100,], exp_df['Appliances'].iloc[:100,])
ax[2,0].tick_params(labelbottom = False, bottom = False)
ax[2,0].set_ylabel('Energy (Wh)')
ax[2,0].set_title('Appliances')
ax[3,0].plot(exp_df['date'].iloc[:100,], exp_df['lights'].iloc[:100,])
ax[3,0].tick_params(labelbottom = False, bottom = False)
ax[3,0].set_ylabel('Energy (Wh)')
ax[3,0].set_title('Lights')
ax[3,0].set_xlabel('Time (mins)')

ax[2,1].plot(exp_df['date'].iloc[:100,], exp_df['T8'].iloc[:100,])
ax[2,1].tick_params(labelbottom = False, bottom = False)
ax[2,1].set_ylabel('Temperature')
ax[3,1].plot(exp_df['date'].iloc[:100,], exp_df['RH_8'].iloc[:100,])
ax[3,1].set_xlabel('Time (mins)')
ax[3,1].set_ylabel('Humidity')
ax[3,1].tick_params(labelbottom = False, bottom = False)
ax[2,1].set_title('Teenager Room')

ax[2,2].plot(exp_df['date'].iloc[:100,], exp_df['T3'].iloc[:100,])
ax[2,2].tick_params(labelbottom = False, bottom = False)
ax[3,2].plot(exp_df['date'].iloc[:100,], exp_df['RH_3'].iloc[:100,])
ax[3,2].tick_params(labelbottom = False, bottom = False)
ax[3,2].set_xlabel('Time (mins)')
ax[2,2].set_title('Parents Room')

ax[2,3].plot(exp_df['date'].iloc[:100,], exp_df['T6'].iloc[:100,])
ax[2,3].tick_params(labelbottom = False, bottom = False)
ax[3,3].plot(exp_df['date'].iloc[:100,], exp_df['RH_6'].iloc[:100,])
ax[3,3].tick_params(labelbottom = False, bottom = False)
ax[3,3].set_xlabel('Time (mins)')
ax[2,3].set_title('Outside Building')

ax[2,4].plot(exp_df['date'].iloc[:100,], exp_df['T7'].iloc[:100,])
ax[2,4].tick_params(labelbottom = False, bottom = False)
ax[3,4].plot(exp_df['date'].iloc[:100,], exp_df['RH_7'].iloc[:100,])
ax[3,4].tick_params(labelbottom = False, bottom = False)
ax[3,4].set_xlabel('Time (mins)')
ax[2,4].set_title('Ironing Room')


plt.tight_layout()

#### Check for Correlations Between Attributes

In [ ]:
#looking at Temperature columns
# regex = '^T' means starts with T
filt = exp_df.filter(regex='^T', axis='columns').corr()
ax = plt.axes()
sns.heatmap(filt.corr(), ax = ax)
ax.set_title('Room Temperature Correlation')
plt.show()

In [ ]:
#looking at Relative Humidity columns
# regex = '^R' means starts with R
filt = exp_df.filter(regex='^R', axis='columns').corr()
ax = plt.axes()
sns.heatmap(filt.corr(), ax = ax)
ax.set_title('Room Relative Humidity Correlation')
plt.show()

#### Check for Potential Transformation Needs

In [ ]:
# rolling mean based on last 5 values
exp_df['rolling_mean_app'] = exp_df['Appliances'].rolling(5).mean()
exp_df['rolling_mean_lights'] = exp_df['lights'].rolling(5).mean()

In [ ]:
fig, ax = plt.subplots(4,1, figsize=(16,9))

ax[0].plot(exp_df['date'], exp_df['rolling_mean_app'])
ax[0].set_title('Appliance Rolling Mean')
ax[0].set_ylabel('Energy (Wh)')
ax[0].tick_params(labelbottom = False, bottom = False)

ax[1].plot(exp_df['date'], exp_df['Appliances'].rolling(5).std())
ax[1].set_title('Appliance Rolling St. Dev')
ax[1].set_ylabel('Energy (Wh)')
ax[1].tick_params(labelbottom = False, bottom = False)

ax[2].plot(exp_df['date'], exp_df['rolling_mean_lights'])
ax[2].set_title('Lights Rolling Mean')
ax[2].set_ylabel('Energy (Wh)')
ax[2].tick_params(labelbottom = False, bottom = False)

ax[3].plot(exp_df['date'], exp_df['lights'].rolling(5).std())
ax[3].set_title('Lights Rolling St. Dev')
ax[3].set_ylabel('Energy (Wh)')
ax[3].tick_params(labelbottom = False, bottom = False)

In [ ]:
exp_df = exp_df.drop(['rolling_mean_lights', 'rolling_mean_app'], axis=1)

#### Document what you have learned

There are many basic points of interest and relationships we are able to see just from our exploratory data analysis. For example, there are no missing data points.If we did have missing data though, we would have been able to reasonably use the value of the point before; this is because the data was taken every ten minutes and the temperature and humidity do not significantly change in that time. When we look at the line graphs we can see the trends of energy usage by appliances and lights in the bottom left hand column. In the right above their graphs, we can see the temperature and relative humidity of the kitchen (chosen since many appliances are in this one room). Based on the graphs of all the rooms, we can see there is an overall spike and decline. Look more closely though, and we see these spikes and declines are not all the same. Interestingly, the graphs of the appliances and lights have a greater amount of spikes and variations then most rooms.<br/><br/>
Upon first glance, these correlation plots reveal a few interesting things. It appears there is not as much of a correlation between temperature and the appliance and lights as there is for relative humidity and appliances and lights. <br/><br/>
Just from these two plots, we hypothesize that relative humidity has a stronger impact on the energy usage of lights rather than appliances. On the flipside, we hypothesize temperature has a stronger impact on the energy usage of appliances than lights.   <br/><br/>
We now must look more closely at relationships that are significant for mulitvariate timeseries forecasting. It is important to see if the data is stationary or not so we will know if we will need to apply transformations to our data. Upon first glance, it is clear the mean and standard deviation of both the appliances and the lights are not stationary - meaning their values are not maintaining a consistent value. This implies transformations will be needed in the next section.

## Prepare the Data

#### Data Cleaning

In [ ]:
exp_df.loc['date'] = pd.to_datetime(exp_df['date'].loc[:])

#### Feature Engineering

In [ ]:
import statsmodels as sm
from statsmodels.tsa.api import VAR, VARMAX
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

**1. Stationarity**

Using Augmented Dickey–Fuller test to check for stationarity.  

*Null hypothesis:* If failed to be rejected, it suggests the time series is not stationary  
*Alternative hypothesis:* The null hypothesis is rejected, it suggests the time series IS stationary.<br/><br/>
If p-values are less than or equal to 0.05 so we can reject the null hypothesis and the data is stationary.

In [ ]:
def adf_test(col, df):
    result = adfuller(df.values)
    # print if not stationary
    if result[1] > 0.05:
        print('Non-stationary column:', col)
        print('p-value:', result[1])
        print('ADF Statistics:', result[0])
        print('p-value:', result[1])
        print('Critical values:')
        for key, value in result[4].items():
            print('\t%s: %.3f' % (key, value))

In [ ]:
exp_df.replace([np.inf, -np.inf], np.nan)
exp_df.dropna(inplace=True)
for col in exp_df.columns[1:]:
    adf_test(col, exp_df[col])

**2. Autocorrelation and Lag Variables**

Autocorrelation is the correlation between a timeseries and the delayed version of itself. ACF is used to show the correlation coefficient against the lag and 0 means there is no correlation. The blue shading is the error bar. This shows the correlation to be at or near zero when the lag is about 25.<br/><br/>
PACF captures a “direct” correlation between time series and a lagged version of itself.

In [ ]:
# Plot the ACF and PACF plots
plot_acf(exp_df['Appliances'], lags=40, title ='Appliance Autocorrelation Function (ACF)')
plot_pacf(exp_df['Appliances'], lags=20, title ='Appliance Partial Autocorrelation Function (PACF)')
plot_acf(exp_df['lights'],lags=40, title = 'Lights Autocorrelation Function (ACF)')
plot_pacf(exp_df['lights'],lags=20, title='Lights Partial Autocorrelation Function (PACF)')

plt.show()

**3. Seasonality, Trend, Residuals**

In [ ]:
decomp = seasonal_decompose(exp_df['Appliances'],model = 'additive',period = 360,extrapolate_trend = 'freq')
fig = decomp.plot()
fig.set_size_inches((16, 9))
fig.tight_layout()
plt.show()

decomp = seasonal_decompose(exp_df['lights'],model = 'additive',period = 360,extrapolate_trend = 'freq')
fig = decomp.plot()
fig.set_size_inches((16, 9))
fig.tight_layout()
plt.show()

#### Feature scaling

Difference model due to non-stationarity

In [ ]:
data = exp_df.drop(['date'], axis=1)

Check stationarity function to make sure that there is no more remaining non-stationary data.

In [ ]:
data.index = exp_df.date
diff_data = data.diff().dropna()
for col in diff_data.columns[1:]:
    adf_test(col, diff_data[col])

#### Feature Selection

Used Granger Casuality to check for significant features

Appliances

In [ ]:
target = diff_data.columns[0]
pred = diff_data.columns[1:]

In [ ]:
results = {}
for predictor in pred:
    data = np.column_stack([diff_data[target], diff_data[predictor]])
    gc_res = grangercausalitytests(data, maxlag=2, verbose=False)
    results[predictor] = gc_res[2][0]['params_ftest'][1]

# Print the results
for predictor, p_value in sorted(results.items(), key=lambda x: x[1]):
    if p_value <= 0.05:
        print(f'{predictor}: {p_value:.4f}')

Lights

In [ ]:
target = diff_data.columns[1]
pred = diff_data.columns[2:]

In [ ]:
results = {}
for predictor in pred:
    data = np.column_stack([diff_data[target], diff_data[predictor]])
    gc_res = grangercausalitytests(data, maxlag=2, verbose=False)
    results[predictor] = gc_res[2][0]['params_ftest'][1]

# Print the results
for predictor, p_value in sorted(results.items(), key=lambda x: x[1]):
    if p_value <= 0.05:
        print(f'{predictor}: {p_value:.4f}')

Create dataframes for Features to be Used

In [ ]:
data_app = diff_data[['Appliances','RH_1', 'T4', 'T3', 'RH_2', 'RH_3', 'T2', 'T1', 'lights','RH_9', 'RH_6', 'RH_7', 'RH_4']]
data_lights = diff_data[['lights', 'T4', 'RH_3', 'RH_1', 'RH_7', 'Visibility', 'T3']]

## Fine-Tune Hyperparameters + Make Model

#### VAR Model - Appliances

Identify lag value

In [ ]:
model = VAR(np.asarray(data_app).astype(float))
model.select_order(20).summary()

Graph with lag value

In [ ]:
model = VARMAX(np.asarray(data_app).astype(float), order=(1,0), trend='n').fit(maxiter=1000)
model.plot_diagnostics(variable=0, figsize=(13,8), lags=24)
plt.gcf().suptitle('Industrial Production - Diagnostics', fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=.9);

Compare at Prediction with data

In [ ]:
predicted_result = model.predict(start=0, end=500)

plt.figure().set_figwidth(17)
plt.plot(data_lights[1:500],color='red')
plt.plot(predicted_result,color='blue')
plt.title('Appliance Energy')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='minor',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=True)
plt.show()
plt.tight_layout()

#### VAR Model - Lights

Identify lag values

In [ ]:
model = VAR(np.asarray(data_lights).astype(float))
model.select_order(20).summary()

Graph with lag value

In [ ]:
model = VARMAX(np.asarray(data_lights).astype(float), order=(1,0), trend='n').fit(maxiter=1000)
model.plot_diagnostics(variable=0, figsize=(13,8), lags=24)
plt.gcf().suptitle('Industrial Production - Diagnostics', fontsize=20)
plt.tight_layout()
plt.subplots_adjust(top=.9);

Compare at Prediction with data

In [ ]:
predicted_result = model.predict(start=0, end=500)

plt.figure().set_figwidth(17)
plt.plot(data_lights[1:500],color='red')
plt.plot(predicted_result,color='blue')
plt.title('Lights Energy')
plt.tick_params(
    axis='x',          # changes apply to the x-axis
    which='minor',      # both major and minor ticks are affected
    bottom=False,      # ticks along the bottom edge are off
    top=False,         # ticks along the top edge are off
    labelbottom=False)
plt.show()
plt.tight_layout()